## **Algoritmo de filtraje no lineal basado en Operador de Koopman aplicado a epidemiología**

### **Capítulo 4: Algoritmo de filtraje no lineal en tiempo discreto**

**Autor: Diego Olguín.**

**Supervisores: Héctor Ramírez y Axel Osses.**

In [262]:
chapter = "chapter4/"
img_path = "img/content/"+chapter

In [263]:
# Librerías generales
import numpy as np
from scipy import stats
from sklearn.gaussian_process.kernels import Matern

# Plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

# Librerías propias
import os

# Dependencias de KKKF
from KKKF.DynamicalSystems import DynamicalSystem
from KKKF.kEDMD import KoopmanOperator
import KKKF.applyKKKF as kkkf

In [264]:
# Ajustar el tamaño de las figuras y los títulos para Plotly
pio.templates.default = 'plotly'
pio.templates['plotly'].layout.update(
    title={'font': {'size': 24}},
    xaxis={'title': {'font': {'size': 20}}},
    yaxis={'title': {'font': {'size': 20}}},
    legend={'font': {'size': 16}},
    margin=dict(l=0, r=0, t=50, b=0),
    width=1000, height=600
);

In [265]:
# Ajustar directorio para las imagénes
root = "/Users/diegoolguin/Koopman_nonlinear_filter"
if not os.getcwd()==root:
    os.chdir(root)

### **Comparación con el filtro de Kalman**

In [266]:
# Función que implementa el filtro de Kalman
def KalmanFilter(y, A, C, Q, R, x0, P0):
    """
    Implementación del filtro de Kalman para un sistema lineal.
    
    Args:
    y: np.array
        Observaciones del sistema.
    A: np.array
        Matriz de transición de estado.
    C: np.array
        Matriz de observación.
    Q: np.array
        Covarianza del ruido del sistema.
    R: np.array 
        Covarianza del ruido de las observaciones.
    x0: np.array    
        Estado inicial.
    P0: np.array
        Covarianza inicial.

    Returns:
    x: np.array
        Estados estimados.
    P: np.array
        Covarianza de los estados estimados.
    """

    # Número de observaciones
    iters = y.shape[1]
    
    # Inicialización de los vectores de estado y covarianza
    x = np.zeros((A.shape[0], iters))
    P = np.zeros((A.shape[0], A.shape[0], iters))
    
    # Inicialización de los estados y covarianza iniciales
    x[:, 0] = x0
    P[:, :, 0] = P0
    
    # Iteración del filtro
    for i in range(1, iters):
        # Predicción
        x[:, i] = A @ x[:, i-1]
        P[:, :, i] = A @ P[:, :, i-1] @ A.T + Q
        
        # Actualización
        K = P[:, :, i] @ C.T @ np.linalg.inv(C @ P[:, :, i] @ C.T + R)
        x[:, i] = x[:, i] + K @ (y[:, i] - C @ x[:, i])
        P[:, :, i] = P[:, :, i] - K @ C @ P[:, :, i]
    
    return x, P

In [267]:
# Función para comparar los resultados del filtro de Kalman y el filtro no lineal
def compare_filters(x0, x0_prior, A, C, Q, R, N, iters):
    """
    Función que compara los resultados del filtro de Kalman y el filtro no lineal.
    
    Args:
    x0: np.array
        Condición inicial verdadera.
    x0_prior: rv_continuous
        Distribución para el estado inicial
    A: np.array
        Matriz de transición de estado.
    C: np.array
        Matriz de observación.
    Q: np.array
        Covarianza del ruido del sistema.
    R: np.array
        Covarianza del ruido de las observaciones.
    N: int
        Dimensión de aproximación de Koopman
    iters: int
        Número de iteraciones.

    Returns:
    x: np.array
        Estados verdaderos del sistema.
    x_kalman: np.array
        Estados estimados por el filtro de Kalman.
    x_kkkf: np.array
        Estados estimados por el filtro no lineal.
    """
    # Funciones de dinámica y observación
    f = lambda x: A @ x
    g = lambda x: C @ x

    # Dimensiones
    nx, ny = A.shape[0], C.shape[0]

    # Distribuciones
    X_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=1*np.eye(nx))
    dyn_dist = stats.multivariate_normal(mean=np.zeros(nx), cov=Q)
    obs_dist = stats.multivariate_normal(mean=np.zeros(ny), cov=R)

    # Sistema dinámico
    dyn = DynamicalSystem(nx, ny, f, g, X_dist, dyn_dist, obs_dist, discrete_time=True)

    # Operador de Koopman
    koopman = KoopmanOperator(kernel_function=Matern(nu=1.5), dynamical_system=dyn)

    # Trayectoria real y observaciones
    x = np.zeros((nx, iters))
    y = np.zeros((ny, iters))
    x[:, 0] = x0
    for i in range(1, iters):
        x[:, i] = A @ x[:, i-1] + np.random.multivariate_normal(np.zeros(nx), Q)
        y[:, i] = C @ x[:, i] + np.random.multivariate_normal(np.zeros(ny), R)

    # Inicialización de las covarianzas iniciales
    P0 = x0_prior.cov

    # Filtro de Kalman
    x_kalman, P_kalman = KalmanFilter(y, A, C, Q, R, x0_prior.mean, P0)

    # Filtro no lineal
    sol_kkkf = kkkf.apply_koopman_kalman_filter(koopman, y.T, x0_prior, N)
    x_kkkf, P_kkkf = sol_kkkf.x_plus.T, sol_kkkf.Px_plus

    return x, x_kalman, x_kkkf

In [268]:
# Condición inicial real
x0 = np.array([1, 1, 1])

# Condición inicial prior
x0_prior = stats.multivariate_normal(mean=np.array([0.8, 1.2, 0.9]), cov=0.01*np.eye(3))

# Matrices de transición y observación
A = np.array([
        [1.01, 0.01, 0.0],
        [0.01, 1.02, -.1],
        [0.0, 0.04, 1.02]
    ])

C = np.array([
    [1, 0, 0],
    [0, 1, 0],
])

# Dimensiones
nx, ny = A.shape[0], C.shape[0]

# Covarianzas del ruido
Q = 0.01*np.eye(nx)
R = 0.01*np.eye(ny)

# Número de dimensiones de aproximación
N = 300

# Número de iteraciones
iters = 30

In [269]:
# Comparación de los filtros
x, x_kalman, x_kkkf = compare_filters(x0, x0_prior, A, C, Q, R, N, iters)

# Gráfico
fig = make_subplots(rows=3, cols=1, subplot_titles=("Estado 1", "Estado 2", "Estado 3"))

fig.add_trace(go.Scatter(x=np.arange(iters), y=x[0, :], mode='lines', name='Real'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_kalman[0, :], mode='lines', name='Kalman'), row=1, col=1)
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_kkkf[0, :], mode='lines', name='KKKF'), row=1, col=1)

fig.add_trace(go.Scatter(x=np.arange(iters), y=x[1, :], mode='lines', name='Real'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_kalman[1, :], mode='lines', name='Kalman'), row=2, col=1)
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_kkkf[1, :], mode='lines', name='KKKF'), row=2, col=1)

fig.add_trace(go.Scatter(x=np.arange(iters), y=x[2, :], mode='lines', name='Real'), row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_kalman[2, :], mode='lines', name='Kalman'), row=3, col=1)
fig.add_trace(go.Scatter(x=np.arange(iters), y=x_kkkf[2, :], mode='lines', name='KKKF'), row=3, col=1)

fig.update_layout(title_text="Comparación de los filtros de Kalman y KKKF")
fig.show()